In [1]:
import sys
import os
import numpy as np
sys.path.insert(0, '../../')

In [2]:
from library.notebook_api.data_loader import CombinedDataLoader

In [3]:
from library.source_data.feature_extractor import AudioFeatureExtractor

In [4]:
#instantiate data 
df = CombinedDataLoader().get_combined_df()
gtza_test_data = df[df.dataset == 'gtzan']

In [5]:
#instantiate extractor
gtza_extract = AudioFeatureExtractor(gtza_test_data.head(5))

In [6]:
#run extraction
gtza_extract.add_audio_data_to_df()

In [8]:
gtza_extract.df['audio_data']#.values[0][2]

track_id
blues.00000.wav    ([0.0073242188, 0.016601562, 0.0076293945, -0....
blues.00001.wav    ([0.0034179688, 0.0043029785, 0.001373291, 0.0...
blues.00002.wav    ([0.019012451, 0.047698975, 0.029418945, -0.01...
blues.00003.wav    ([-0.013000488, -0.03060913, -0.036071777, -0....
blues.00004.wav    ([-0.0063171387, -0.009277344, -0.008331299, -...
Name: audio_data, dtype: object

In [14]:
#checkout features for a record 
sample_features = (gtza_extract
 .extract_features(gtza_extract.df['audio_data'].iloc[0][0], gtza_extract.df['audio_data'].iloc[0][1])
 
 )

In [ ]:
np.array(list(sample_features.values()))

In [ ]:
sample_features['spectral_bandwidth_mean']

In [10]:
test_df = gtza_extract.df.copy()

In [11]:
test_df['features'] = sample_features

In [ ]:
test_df['features']